In [1]:
import numpy as np
from sklearn.model_selection import train_test_split, train_test_split
import pandas as pd

from utils.data import extract_eeg_data, segment_data, kfold_split
from utils.labels import get_pss_labels, get_stai_labels
from utils.valid_recs import get_valid_recs
from features import all_features, hjorth_features
from classifiers import knn_classification, svm_classification, cnn_classification
import utils.variables as v

In [2]:
valid_recs = get_valid_recs(data_type='ica')

x_dict = extract_eeg_data(valid_recs, data_type='ica')


(8, 75072)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75168)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75584)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75584)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75024)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75040)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75536)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0

ERROR:root:1) Failed to read data for recording P006_S002_001
ERROR:root:1) Failed to read data for recording P006_S002_002


(8, 75152)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75040)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 82688)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75664)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75200)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75424)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75328)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0

ERROR:root:1) Failed to read data for recording P028_S001_001
ERROR:root:1) Failed to read data for recording P028_S001_002


(8, 75424)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75312)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75072)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75168)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75584)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75584)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0 ... 74999 =      0.000 ...   299.996 secs
Ready.
(8, 75024)
(8, 75000)
Creating RawArray with float64 data, n_channels=8, n_times=75000
    Range : 0

In [3]:
print(x_dict)

{'P001_S001_001': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P001_S001_002': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P001_S002_001': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P001_S002_002': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P002_S001_001': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P002_S001_002': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P002_S002_001': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P002_S002_002': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P003_S001_001': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P003_S001_002': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P003_S002_001': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P003_S002_002': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P004_S001_001': <RawArray | 8 x 75000 (300.0 s), ~4.6 MB, data loaded>, 'P004_S001_002': <RawArray | 8 x 75000 (300.0 s), 